In [9]:

import os
import requests
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
import re

print("=== 네이버 쇼핑 API 향수 가격 조회 에이전트 ===\n")

# 🔑 Colab 환경변수에서 불러오기
openai_api_key = os.getenv("OPENAI_API_KEY")
naver_client_id = os.getenv("NAVER_CLIENT_ID")
naver_client_secret = os.getenv("NAVER_CLIENT_SECRET")

if not openai_api_key or not naver_client_id or not naver_client_secret:
    raise ValueError("❌ Colab 환경변수에 API 키가 등록되어 있지 않습니다.")

# ✅ OpenAI LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=openai_api_key)
print("✅ OpenAI LLM 초기화 완료!\n")


# ========================================
# 가격 조회 툴 정의
# ========================================
@tool
def price_agent_tool(user_query: str):
    """네이버 쇼핑 API를 사용해 향수 가격을 조회하는 툴"""
    url = "https://openapi.naver.com/v1/search/shop.json"
    headers = {
        "X-Naver-Client-Id": naver_client_id,
        "X-Naver-Client-Secret": naver_client_secret
    }
    params = {"query": user_query, "display": 5, "sort": "sim"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        if data and "items" in data and len(data["items"]) > 0:
            products = []
            for item in data["items"][:3]:
                def remove_html_tags(text): return re.sub(r'<[^>]+>', '', text)
                products.append({
                    "title": remove_html_tags(item.get("title", "")),
                    "lprice": item.get("lprice", "정보 없음"),
                    "hprice": item.get("hprice", "정보 없음"),
                    "mallName": item.get("mallName", "정보 없음"),
                    "link": item.get("link", "정보 없음"),
                    "brand": item.get("brand", "정보 없음"),
                })
            return {"status": "success", "query": user_query, "products": products}
        else:
            return {"status": "no_results", "query": user_query}
    else:
        return {"status": "error", "message": f"API 오류 {response.status_code}"}


# ✅ 결과 포맷팅
def format_price_result(result):
    if result["status"] == "success":
        output = f"🔍 '{result['query']}' 검색 결과:\n\n"
        for i, p in enumerate(result["products"], 1):
            output += f"📦 {i}. {p['title']}\n"
            if p['lprice'] != "0":
                output += f"   💰 가격: {int(p['lprice']):,}원\n"
            output += f"   🏪 판매처: {p['mallName']}\n"
            output += f"   🔗 링크: {p['link']}\n\n"
        return output
    elif result["status"] == "no_results":
        return f"😔 '{result['query']}'에 대한 검색 결과가 없습니다."
    else:
        return f"❌ 오류: {result.get('message', '알 수 없는 오류')}"


# ✅ LangGraph 에이전트 구성
def create_price_agent():
    workflow = StateGraph(dict)

    def search_node(state):
        user_query = state["user_query"]
        search_result = price_agent_tool.invoke(user_query)
        return {"search_result": search_result, "formatted_output": format_price_result(search_result)}

    workflow.add_node("search", search_node)
    workflow.set_entry_point("search")
    workflow.add_edge("search", END)
    return workflow.compile()


def search_perfume_price(query):
    app = create_price_agent()
    result = app.invoke({"user_query": query})
    return result["formatted_output"]


# ========================================
# ✅ 자동 실행: 샘플 검색어들 테스트
# ========================================
sample_queries = ["샤넬 웜코튼 향수추천좀"]

print("\n🚀 샘플 검색 실행:\n")
for q in sample_queries:
    print(f"🔍 검색어: {q}")
    print(search_perfume_price(q))
    print("="*50 + "\n")


=== 네이버 쇼핑 API 향수 가격 조회 에이전트 ===

✅ OpenAI LLM 초기화 완료!


🚀 샘플 검색 실행:

🔍 검색어: 샤넬 웜코튼 향수추천좀
😔 '샤넬 웜코튼 향수추천좀'에 대한 검색 결과가 없습니다.

